<a href="https://colab.research.google.com/github/LarsHadidi/PRONTO/blob/mathprogram/mp/PDP-GEO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Progressive Dinner Party: Geometric Program

# Method: Solving clustered subsets

Clustering is the task of grouping a set of objects in such a way that objects in the same group (called a cluster) are more similar to each other than to those in other groups.

<img alt="clusters" src="https://raw.githubusercontent.com/benedekrozemberczki/awesome-community-detection/master/coms.png" width="25%"/>

## Imports

In [5]:
import numpy as np
import bokeh.palettes
import bokeh.plotting as bkh
from sklearn import datasets, cluster, neighbors
bkh.output_notebook()

## Data

In [2]:
data, _ = datasets.make_blobs(n_samples=3000, centers=3, cluster_std=0.5)
data = np.dot(data, [[-0.7, 0.5], [-0.2, -0.7]])
X = data[:,0]
Y = data[:,1]


bkh.output_notebook()
p = bkh.figure(width=800, height=800)
p.scatter(X, Y, fill_color='blue', alpha=0.2, size=10)
bkh.show(p)

##Clustering

In [3]:
clusterer = cluster.MeanShift()
labels = clusterer.fit_predict(data)
centroids = {label: np.mean(data[labels==label,:], axis=0) for label in set(labels) if label != -1}

cmap = bokeh.palettes.all_palettes['Paired'][max(3,len(set(labels)))]

source = bkh.ColumnDataSource(data=dict(
    x=X,
    y=Y,
    color=[list(cmap)[i] for i in labels],
    label=labels
))

p = bkh.figure(width=800, height=800)
p.scatter(source=source, fill_color='color', size=10, legend_group='label')
p.scatter(np.vstack(list(centroids.values()))[:,0],np.vstack(list(centroids.values()))[:,1], fill_color='red', marker='triangle_pin', size=25)
bkh.show(p)

In [6]:
dispersion = {y: np.sqrt(np.linalg.norm(np.var(data[labels == y,:], axis=0))) for y in set(labels)}
kde = {}
Z = {}
for y in set(labels):
  kde[y] = neighbors.KernelDensity(bandwidth=dispersion[y]).fit(data[labels == y])
  Z[y] = np.exp(kde[y].score_samples(data))

carrier_stack = list(set(labels))
distance = np.empty(shape=(len(data),len(Z)))
for c in Z:
  distance[:,c] = - Z[c]

while carrier_stack:
  current_cluster = carrier_stack.pop()
  n = sum(labels==current_cluster)
  r = 3 - n % 3 if n % 3 != 0 else 0
  if r != 0:
    if len(carrier_stack) > 0:
      d = np.array([distance[idx,current_cluster] if cluster in carrier_stack else np.Infinity for idx,cluster in enumerate(labels)])
      nearest_idx = np.argsort(d)[:r]
      labels[nearest_idx] = current_cluster
    else:
      d = np.array([distance[idx,current_cluster] if cluster == current_cluster else np.negative(np.Infinity) for idx,cluster in enumerate(labels)])
      dropout_idx = np.argsort(d)[-r:]
      labels[dropout_idx] = -1

bkh.output_notebook()
cmap = bokeh.palettes.all_palettes['Paired'][max(3,len(set(labels)))]

source = bkh.ColumnDataSource(data=dict(
    x=X,
    y=Y,
    d=[str(x) for x in distance],
    color=[list(cmap)[i] for i in labels],
    label=labels
))

p = bkh.figure(width=800, height=800, tooltips=[('Distance', '@d')], tools='pan,wheel_zoom,save,reset', active_drag='pan', active_scroll='wheel_zoom')
p.scatter(source=source, fill_color='color', size=10, legend_group='label')

bkh.show(p)

print({c: sum(labels==c) for c in set(labels)})

{0: 996, 1: 1002, 2: 1002}
